In [1]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
from logger import Logger 
from datetime import datetime
import tqdm 
import pandas as pd
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [2]:
# Prepare source dataset 

source_name = 'data/penalty/Penalty_size_{size}_penalty_{penalty}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'penalty' : 'VLM',
    'count' : 50,
    'budget_fracs' : [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], 
    'shots' : 1000, 
    'stepsize' : 0.25, 
    'accepterr': [0.01,0.05,0.01], 
    'iterations_limit' : 1000, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'MA-PIA'
}

template = pd.read_csv('data/template_penalty.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], penalty = Config['penalty'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()


log_file, id = log._load_logfile()


Logfile data/penalty/Penalty_size_10_penalty_VLM_ansatz_MA-PIA_stepsize_0.25_shots_1000.csv already exists


In [3]:

for i in tqdm.tqdm(range(Config['count']), desc = f"Generating matrices with size {Config['size']}"):
    matrix = np.random.uniform(size = (Config['size'], Config['size']), low = -1, high = 1) 
    matrix = (matrix.T + matrix) / 2
    for budget_frac in Config['budget_fracs']: 
    
        id += 1
        
        current_datetime = datetime.now()
        data = { 
            'Id (int)' : id, 
            'Valid (int)' : 1, 
            'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            'Matrix (mat)' : matrix,
            'Size (int)' : Config['size'],
            'Type (str)' : 'Penalty',
            'Rank (int)' : -1,
            'Density (flt)' : -1.0,
            'Stiffness (flt)' : -1.0,
            'Penalty (str)' : Config['penalty'],
            'Budget (flt)' : Config['size'] * budget_frac,
            'Test_solver (str)' : '-',
            'Test (flt)' : -1.0,
            'Test_state (vec)' : [],
            'Test_iterations (int)' : -1,
            'Test_time (flt)' : -1.0,
            'Train_solver (str)' : '-',
            'Preprocessing (str)' : '-',
            'Postprocessing (str)' : '-',
            'Train (flt)' : -1.0,
            'Train_state (vec)' : [],
            'Train_iterations (int)' : -1, 
            'Train_iterations_1 (int)' : -1,
            'Train_iterations_2 (int)' : -1,
            'Train_iterations_3 (int)' : -1,
            'Train_shots (int)': -1, 
            'Train_cost (flt)' : -1.0,
            'Train_time (flt)' : -1.0, 
            'Flags (str)' : '',
            'Baren (int)' : -1
        }
        
        log.log(data)

    
display(log.read(1))

Generating matrices with size 10:   8%|▊         | 4/50 [00:04<00:50,  1.10s/it]


KeyboardInterrupt: 

In [4]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])
prep = Preparator()

In [6]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        #print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''
            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h, constrain = True, k = data['Budget (flt)'])
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
        print(f'Matrix : {np.array(data['Matrix (mat)'])} | Budget : {data['Budget (flt)']} | Penalty : {Config['penalty']}')

        
        Q = data['Matrix (mat)']
        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

In [7]:

ids = range (1,351)
cores = 10
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    #display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



Matrix : [[-0.25181119 -0.1254799  -0.33913343 -0.54804844  0.48575397  0.00416723
  -0.13161105  0.11885091  0.4378535  -0.30084214]
 [-0.1254799   0.31168767  0.49760748 -0.10016909  0.30296345 -0.30083645
  -0.31807456 -0.4082926   0.76215545  0.72714599]
 [-0.33913343  0.49760748 -0.94548178 -0.69276835  0.22712303  0.30864774
  -0.52329014  0.44083197 -0.04201562 -0.36496583]
 [-0.54804844 -0.10016909 -0.69276835 -0.34846136 -0.12848566  0.44201181
  -0.36708818  0.14365261 -0.3802448   0.35051104]
 [ 0.48575397  0.30296345  0.22712303 -0.12848566  0.70924828 -0.40359694
   0.04008263  0.39007575  0.12699196 -0.38327257]
 [ 0.00416723 -0.30083645  0.30864774  0.44201181 -0.40359694 -0.95180102
  -0.16073068  0.02205378  0.50390329  0.00684598]
 [-0.13161105 -0.31807456 -0.52329014 -0.36708818  0.04008263 -0.16073068
  -0.27054234  0.03755638  0.56348647 -0.464205  ]
 [ 0.11885091 -0.4082926   0.44083197  0.14365261  0.39007575  0.02205378
   0.03755638 -0.80376627  0.57508795  0.4

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, lis

In [3]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        #Q = line['Matrix (mat)']
        
        trained_data = line.copy()
        Q = line['Matrix (mat)']
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False, constrain = True, k = line['Budget (flt)'], penalty = Config['penalty'])
        print(f'Trained id = {line['Id (int)']} with train {train} and budget {np.sum(np.where(train_bitstring > 0, 1, 0))} while test budget is {line['Budget (flt)']} | train_bitstring : {train_bitstring}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [ ]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 10

solvers = [] 

depth = 3

ids = np.array(range(1, 351))
ids = ids.reshape(cores, len(ids) // cores)

template = pd.read_csv('data/template_penalty.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = Config['ansatz'], gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))



log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], penalty = Config['penalty'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()

data = [[log.read (id) for id in subrange] for subrange in ids]
#print(train_dataset)
# data = [[log.read(id) for id in range (id_start, id_start + len(train_dataset) // cores)] for id_start in range (1, len(train_dataset), len(train_dataset) // cores) ]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)


NameError: name 'processed_data' is not defined